# Getting started

### CLEF 2025 - CheckThat! Lab  - Task 4 Scientific Web Discourse - Subtask 4b (Scientific Claim Source Retrieval)

This notebook enables participants of subtask 4b to quickly get started. It includes the following:
- Code to upload data, including:
    - code to upload the collection set (CORD-19 academic papers' metadata)
    - code to upload the query set (tweets with implicit references to CORD-19 papers)
- Code to run a baseline retrieval model (BM25)
- Code to evaluate the baseline model

Participants are free to use this notebook and add their own models for the competition.

# 1) Importing data

In [5]:
%pip install numpy pandas

#!/usr/bin/env python3
import numpy as np
import pandas as pd
import pathlib

Defaulting to user installation because normal site-packages is not writeable
  Using cached pandas-2.2.3-cp39-cp39-macosx_10_9_x86_64.whl (12.6 MB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## 1.a) Import the collection set
The collection set contains metadata of CORD-19 academic papers.

The preprocessed and filtered CORD-19 dataset is available on the Gitlab repository here: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b

Participants should first download the file then upload it on the Google Colab session with the following steps.


In [ ]:
# 1) Download the collection set from the Gitlab repository: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b
# 2) Drag and drop the downloaded file to the "Files" section (left vertical menu on Colab)
# 3) Modify the path to your local file path
root_dir = pathlib.Path().cwd().parent.resolve()
test_path        = root_dir / "data" / "subtask4b_query_tweets_test.tsv"
train_path        = root_dir / "data" / "subtask4b_query_tweets_train.tsv"
dev_path        = root_dir / "data" / "subtask4b_query_tweets_dev.tsv"
collection_path  = root_dir / "data" / "subtask4b_collection_data.pkl"
paper_ids_path   = root_dir / "models" / "paper_ids.pkl"
faiss_index_path = root_dir / "models" / "paper_index.faiss"
model_dir        = root_dir / "models"

PATH_COLLECTION_DATA = collection_path #MODIFY PATH



/Users/agonsylejmani/Downloads/AIR-1/Representation Learning (Dense Retrieval)/NLP representation learning/data/subtask4b_collection_data.pkl


In [9]:
df_collection = pd.read_pickle(PATH_COLLECTION_DATA)

In [10]:
df_collection.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7718 entries, 162 to 1056448
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   cord_uid          7718 non-null   object        
 1   source_x          7718 non-null   object        
 2   title             7718 non-null   object        
 3   doi               7677 non-null   object        
 4   pmcid             4959 non-null   object        
 5   pubmed_id         6233 non-null   object        
 6   license           7718 non-null   object        
 7   abstract          7718 non-null   object        
 8   publish_time      7715 non-null   object        
 9   authors           7674 non-null   object        
 10  journal           6668 non-null   object        
 11  mag_id            0 non-null      float64       
 12  who_covidence_id  528 non-null    object        
 13  arxiv_id          20 non-null     object        
 14  label             7718 n

In [11]:
df_collection.head()

,cord_uid,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,label,time,timet
162,umvrwgaw,PMC,Professional and Home-Made Face Masks Reduce E...,10.1371/journal.pone.0002618,PMC2440799,18612429,cc-by,BACKGROUND: Governments are preparing for a po...,2008-07-09,"van der Sande, Marianne; Teunis, Peter; Sabel,...",PLoS One,NaN,NaN,NaN,umvrwgaw,2008-07-09,1215561600
611,spiud6ok,PMC,The Failure of R (0),10.1155/2011/527610,PMC3157160,21860658,cc-by,"The basic reproductive ratio, R (0), is one of...",2011-08-16,"Li, Jing; Blakeley, Daniel; Smith?, Robert J.",Comput Math Methods Med,NaN,NaN,NaN,spiud6ok,2011-08-16,1313452800
918,aclzp3iy,PMC,Pulmonary sequelae in a patient recovered from...,10.4103/0970-2113.99118,PMC3424870,22919170,cc-by-nc-sa,The pandemic of swine flu (H1N1) influenza spr...,2012,"Singh, Virendra; Sharma, Bharat Bhushan; Patel...",Lung India,NaN,NaN,NaN,aclzp3iy,2012-01-01,1325376000
993,ycxyn2a2,PMC,What was the primary mode of smallpox transmis...,10.3389/fcimb.2012.00150,PMC3509329,23226686,cc-by,The mode of infection transmission has profoun...,2012-11-29,"Milton, Donald K.",Front Cell Infect Microbiol,NaN,NaN,NaN,ycxyn2a2,2012-11-29,1354147200
1053,zxe95qy9,PMC,"Lessons from the History of Quarantine, from P...",10.3201/eid1902.120312,PMC3559034,23343512,no-cc,"In the new millennium, the centuries-old strat...",2013-02-03,"Tognotti, Eugenia",Emerg Infect Dis,NaN,NaN,NaN,zxe95qy9,2013-02-03,1359849600


## 1.b) Import the query set

The query set contains tweets with implicit references to academic papers from the collection set.

The preprocessed query set is available on the Gitlab repository here: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b

Participants should first download the file then upload it on the Google Colab session with the following steps.

In [16]:
# 1) Download the query tweets from the Gitlab repository: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b?ref_type=heads
# 2) Drag and drop the downloaded file to the "Files" section (left vertical menu on Colab)
# 3) Modify the path to your local file path

PATH_QUERY_TRAIN_DATA = train_path #MODIFY PATH
PATH_QUERY_DEV_DATA = dev_path #MODIFY PATH

In [17]:
df_query_train = pd.read_csv(PATH_QUERY_TRAIN_DATA, sep = '\t')
df_query_dev = pd.read_csv(PATH_QUERY_DEV_DATA, sep = '\t')

In [18]:
df_query_train.head()

,post_id,tweet_text,cord_uid
0,0,Oral care in rehabilitation medicine: oral vul...,htlvpvz5
1,1,this study isn't receiving sufficient attentio...,4kfl29ul
2,2,"thanks, xi jinping. a reminder that this study...",jtwb17u8
3,3,Taiwan - a population of 23 million has had ju...,0w9k8iy1
4,4,Obtaining a diagnosis of autism in lower incom...,tiqksd69


In [19]:
df_query_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12853 entries, 0 to 12852
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_id     12853 non-null  int64 
 1   tweet_text  12853 non-null  object
 2   cord_uid    12853 non-null  object
dtypes: int64(1), object(2)
memory usage: 301.4+ KB


In [20]:
df_query_dev.head()

,post_id,tweet_text,cord_uid
0,16,covid recovery: this study from the usa reveal...,3qvh482o
1,69,"""Among 139 clients exposed to two symptomatic ...",r58aohnu
2,73,I recall early on reading that researchers who...,sts48u9i
3,93,You know you're credible when NIH website has ...,3sr2exq9
4,96,Resistance to antifungal medications is a grow...,ybwwmyqy


In [21]:
df_query_dev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_id     1400 non-null   int64 
 1   tweet_text  1400 non-null   object
 2   cord_uid    1400 non-null   object
dtypes: int64(1), object(2)
memory usage: 32.9+ KB


# 2) Running the baseline
The following code runs a BM25 baseline.


In [27]:
%pip install rank_bm25
from rank_bm25 import BM25Okapi


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [28]:
# Create the BM25 corpus
corpus = df_collection[:][['title', 'abstract']].apply(lambda x: f"{x['title']} {x['abstract']}", axis=1).tolist()
cord_uids = df_collection[:]['cord_uid'].tolist()
tokenized_corpus = [doc.split(' ') for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

In [29]:
for i, tokens in enumerate(tokenized_corpus[:3]):
    print(f"Document {i}: {tokens}")


Document 0: ['Professional', 'and', 'Home-Made', 'Face', 'Masks', 'Reduce', 'Exposure', 'to', 'Respiratory', 'Infections', 'among', 'the', 'General', 'Population', 'BACKGROUND:', 'Governments', 'are', 'preparing', 'for', 'a', 'potential', 'influenza', 'pandemic.', 'Therefore', 'they', 'need', 'data', 'to', 'assess', 'the', 'possible', 'impact', 'of', 'interventions.', 'Face-masks', 'worn', 'by', 'the', 'general', 'population', 'could', 'be', 'an', 'accessible', 'and', 'affordable', 'intervention,', 'if', 'effective', 'when', 'worn', 'under', 'routine', 'circumstances.', 'METHODOLOGY:', 'We', 'assessed', 'transmission', 'reduction', 'potential', 'provided', 'by', 'personal', 'respirators,', 'surgical', 'masks', 'and', 'home-made', 'masks', 'when', 'worn', 'during', 'a', 'variety', 'of', 'activities', 'by', 'healthy', 'volunteers', 'and', 'a', 'simulated', 'patient.', 'PRINCIPAL', 'FINDINGS:', 'All', 'types', 'of', 'masks', 'reduced', 'aerosol', 'exposure,', 'relatively', 'stable', 'over

In [30]:
text2bm25top = {}

def get_top_cord_uids(query):
  if query in text2bm25top.keys():
      return text2bm25top[query]
  else:
      tokenized_query = query.split(' ')
      doc_scores = bm25.get_scores(tokenized_query)
      indices = np.argsort(-doc_scores)[:5]
      bm25_topk = [cord_uids[x] for x in indices]

      text2bm25top[query] = bm25_topk
      return bm25_topk


In [31]:
# Retrieve topk candidates using the BM25 model
df_query_train['bm25_topk'] = df_query_train['tweet_text'].apply(lambda x: get_top_cord_uids(x))
df_query_dev['bm25_topk'] = df_query_dev['tweet_text'].apply(lambda x: get_top_cord_uids(x))

In [25]:
df_query_train.head()
#df_query_dev.head()

,post_id,tweet_text,cord_uid,bm25_topk
0,0,Oral care in rehabilitation medicine: oral vul...,htlvpvz5,"[htlvpvz5, h7hj64q5, rwgqkow3, dbgtslc8, am11y..."
1,1,this study isn't receiving sufficient attentio...,4kfl29ul,"[2cwvga0k, 33znyrn8, st3fyb64, 74dw6emg, b68c8..."
2,2,"thanks, xi jinping. a reminder that this study...",jtwb17u8,"[8hkxbxz9, w98847ai, jtwb17u8, iy1enazk, pg0l9..."
3,3,Taiwan - a population of 23 million has had ju...,0w9k8iy1,"[32ua8wb6, zxe95qy9, 30pl5tx3, l4y7v729, b97ac..."
4,4,Obtaining a diagnosis of autism in lower incom...,tiqksd69,"[tiqksd69, k7smwz6w, b0dzhsrh, yc7cvbii, iiafu..."


# 3) Evaluating the baseline
The following code evaluates the BM25 retrieval baseline on the query set using the Mean Reciprocal Rank score (MRR@5).

In [35]:
# Evaluate retrieved candidates using MRR@k
def get_performance_mrr(data, col_gold, col_pred, list_k = [1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(lambda x: (1/([i for i in x[col_pred][:k]].index(x[col_gold]) + 1) if x[col_gold] in [i for i in x[col_pred][:k]] else 0), axis=1)
        #performances.append(data["in_topx"].mean())
        d_performance[k] = data["in_topx"].mean()
    return d_performance


In [36]:
results_train = get_performance_mrr(df_query_train, 'cord_uid', 'bm25_topk')
results_dev = get_performance_mrr(df_query_dev, 'cord_uid', 'bm25_topk')
# Printed MRR@k results in the following format: {k: MRR@k}
print(f"Results on the train set: {results_train}")
print(f"Results on the dev set: {results_dev}")

Results on the train set: {1: np.float64(0.5081303975725512), 5: np.float64(0.5509777224513084), 10: np.float64(0.5509777224513084)}
Results on the dev set: {1: np.float64(0.5057142857142857), 5: np.float64(0.5522738095238094), 10: np.float64(0.5522738095238094)}


# 4) Exporting results to prepare the submission on Codalab

In [38]:
# %% [markdown]
# ## 4) LOAD, MERGE & EVALUATE DENSE RUNS
import json

RUN_TRAIN  = root_dir / "dense_train_only.tsv"
RUN_DEV    = root_dir / "dense_dev_only.tsv"
OUT_SUBMIT = root_dir / "notebooks" / "predictions.tsv"

# 4.1 load each run into its own DF
df_run_tr = pd.read_csv(RUN_TRAIN, sep="\t", names=["post_id","preds_json"], dtype=str)
df_run_tr["preds"] = df_run_tr["preds_json"].map(json.loads)

df_run_de = pd.read_csv(RUN_DEV, sep="\t", names=["post_id","preds_json"], dtype=str)
df_run_de["preds"] = df_run_de["preds_json"].map(json.loads)

# --- make every 'post_id' a *string* ---
for df in (df_query_train, df_query_dev, df_run_tr, df_run_de):
    df["post_id"] = df["post_id"].astype(str)

# now your merges succeed
df_e_train = df_query_train.merge(
    df_run_tr[["post_id", "preds"]],
    on="post_id", how="left", suffixes=(None, "_dense")
)
df_e_dev = df_query_dev.merge(
    df_run_de[["post_id", "preds"]],
    on="post_id", how="left", suffixes=(None, "_dense")
)

# 4.3 normalize preds so it's always a list; drop stray preds_* columns
for df_e in (df_e_train, df_e_dev):
    drop_cols = [c for c in df_e.columns if c.startswith("preds_") and c != "preds"]
    df_e.drop(columns=drop_cols, errors="ignore", inplace=True)
    df_e["preds"] = df_e["preds"].apply(lambda x: x if isinstance(x, list) else [])

print("Columns in df_e_train:", df_e_train.columns.tolist())
print("Columns in df_e_dev:  ", df_e_dev.columns.tolist())

# 4.4 compute Dense-retrieval MRR@1,5,10
print("Dense MRR@k on TRAIN:", get_performance_mrr(df_e_train, "cord_uid", "preds"))
print("Dense MRR@k on DEV:  ", get_performance_mrr(df_e_dev,   "cord_uid", "preds"))

# %% [markdown]
# ## 5) WRITE OUT `predictions.tsv` FOR CODALAB

# by default submit your dense dev run:
df_query_dev["preds"] = df_e_dev["preds"]

# if you'd rather submit BM25, uncomment:
# df_dev["preds"] = df_dev["bm25_top5"]

df_query_dev[["post_id","preds"]].to_csv(OUT_SUBMIT, sep="\t", index=False)
print("✅ Wrote:", OUT_SUBMIT)


Columns in df_e_train: ['post_id', 'tweet_text', 'cord_uid', 'bm25_topk', 'in_topx', 'preds']
Columns in df_e_dev:   ['post_id', 'tweet_text', 'cord_uid', 'bm25_topk', 'in_topx', 'preds']
Dense MRR@k on TRAIN: {1: np.float64(0.5618143624056641), 5: np.float64(0.6398207941077311), 10: np.float64(0.6398207941077311)}
Dense MRR@k on DEV:   {1: np.float64(0.5328571428571428), 5: np.float64(0.599047619047619), 10: np.float64(0.599047619047619)}
✅ Wrote: /Users/agonsylejmani/Downloads/AIR-1/Representation Learning (Dense Retrieval)/NLP representation learning/notebooks/predictions.tsv
